In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

np.set_printoptions(precision=3, suppress=True)
data_train=pd.read_csv("train.csv")
data_train=data_train.drop(['pickup_time','accept_time','allot_time','order_date','order_time','reassignment_reason','reassigned_order','rider_id','reassignment_method','cancelled_time','undelivered_orders','delivered_time'],axis=1)
labels=list(data_train.head(0))
x_id=data_train["order_id"]
y=data_train["cancelled"]
data_train=data_train.drop(["cancelled","order_id"],axis=1)

In [2]:
data_train.head()

,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,lifetime_order_count,session_time
0,1.5666,2.65,46.0,46.0,621.0,NaN
1,2.5207,2.76,8.0,8.0,105.0,3.266667
2,2.2074,4.80,1.0,1.0,66.0,9.816667
3,2.1894,6.38,1.0,1.0,127.0,17.533333
4,2.7870,4.01,34.0,34.0,84.0,1.350000


In [3]:
data_train.to_numpy()
data_train=np.nan_to_num(data_train)
data_train.shape


(450000, 6)

In [4]:
y.to_numpy()
train_examples=data_train
train_labels=y


In [5]:
data_test=pd.read_csv("test.csv")
data_test=data_test.drop(['accept_time','allot_time','order_date','order_time','reassignment_reason','reassigned_order','rider_id','reassignment_method','undelivered_orders'],axis=1)
labels=list(data_test.head(0))
test_id=data_test["order_id"]
y=data_test["cancelled"]
data_test=data_test.drop(["cancelled","order_id"],axis=1)
data_test.to_numpy()
data_test=np.nan_to_num(data_test)
y.to_numpy()
test_examples=data_test
test_labels=y

In [6]:
SHUFFLE_BUFFER = 500
BATCH_SIZE = 450
tf.convert_to_tensor(train_examples)

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-02-02 11:11:36.895624: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-02-02 11:11:36.896317: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


<tf.Tensor: shape=(450000, 6), dtype=float64, numpy=
array([[   1.567,    2.65 ,   46.   ,   46.   ,  621.   ,    0.   ],
       [   2.521,    2.76 ,    8.   ,    8.   ,  105.   ,    3.267],
       [   2.207,    4.8  ,    1.   ,    1.   ,   66.   ,    9.817],
       ...,
       [   1.594,    1.61 ,   28.   ,   28.   , 1488.   ,  204.15 ],
       [   2.894,    4.68 ,   72.   ,   72.   ,  105.   ,   65.583],
       [   1.893,    0.09 ,   30.   ,   30.   ,  108.   ,  212.   ]])>

In [7]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(train_examples)

2022-02-02 11:11:39.881290: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-02-02 11:11:39.884199: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-02-02 11:11:39.944361: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-02 11:11:39.955203: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [24]:
def get_basic_model():
  model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(10, activation='tanh'),
    tf.keras.layers.Dense(6,activation= 'relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
  ])
  #opt = tf.keras.optimizers.SGD(lr=0.01)
  model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [26]:
model = get_basic_model()
model.fit(train_examples, train_labels, epochs=1, batch_size=BATCH_SIZE)

  29/1000 [..............................] - ETA: 3s - loss: 0.7701 - accuracy: 0.5477

2022-02-02 11:18:28.652352: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1000/1000 [==============================] - 4s 3ms/step - loss: 0.2153 - accuracy: 0.9290


In [9]:
'''train_dataset = tf.data.Dataset.from_tensor_slices((train_examples, train_labels))
train_batches = train_dataset.shuffle(1000).batch(450)

model = get_basic_model()
model.fit(train_batches, epochs=15)'''

/Users/aman/miniforge3/lib/python3.9/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
/Users/aman/miniforge3/lib/python3.9/site-packages/keras/backend.py:4993: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(
2022-02-02 11:11:57.785250: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 1/15
1000/1000 [==============================] - 5s 5ms/step - loss: 0.1827 - accuracy: 0.9567
Epoch 2/15
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0760 - accuracy: 0.9884
Epoch 3/15
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0700 - accuracy: 0.9884
Epoch 4/15
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0681 - accuracy: 0.9884
Epoch 5/15
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0671 - accuracy: 0.9884
Epoch 6/15
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0665 - accuracy: 0.9884
Epoch 7/15
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0660 - accuracy: 0.9884
Epoch 8/15
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0656 - accuracy: 0.9884
Epoch 9/15
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0653 - accuracy: 0.9884
Epoch 10/15
1000/1000 [==============================] - 5s 5ms/step - loss: 0.065

In [27]:
tf.convert_to_tensor(test_examples)
test_dataset = tf.data.Dataset.from_tensor_slices((test_examples, test_labels))
model.evaluate(test_examples,test_labels)

  51/4527 [..............................] - ETA: 13s - loss: 0.0169 - accuracy: 1.0000

2022-02-02 11:18:36.112471: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


4527/4527 [==============================] - 13s 3ms/step - loss: 0.0186 - accuracy: 1.0000


[0.0186103917658329, 1.0]

In [28]:
ans=model.predict(test_examples)

2022-02-02 11:18:52.178502: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [29]:
ans

array([[0.011],
       [0.023],
       [0.009],
       ...,
       [0.018],
       [0.014],
       [0.023]], dtype=float32)

In [31]:
min(ans)

array([0.005], dtype=float32)

In [142]:
min(ans)

array([0.014], dtype=float32)

In [32]:
ans.shape

(144844, 1)

In [49]:
sum(ans)/len(ans)

array([0.018])

In [50]:
for i, predicted in enumerate(ans):
    if predicted[0] > 0.018:
        print ("bigger than 0.25")
        #assign i to class 1
    else:
        print ("smaller than 0.25")
        #assign i to class 0
ans.dtype

smaller than 0.25
bigger than 0.25
smaller than 0.25
smaller than 0.25
smaller than 0.25
bigger than 0.25
smaller than 0.25
smaller than 0.25
bigger than 0.25
smaller than 0.25
smaller than 0.25
bigger than 0.25
bigger than 0.25
smaller than 0.25
smaller than 0.25
bigger than 0.25
smaller than 0.25
bigger than 0.25
smaller than 0.25
smaller than 0.25
bigger than 0.25
smaller than 0.25
smaller than 0.25
smaller than 0.25
smaller than 0.25
smaller than 0.25
smaller than 0.25
bigger than 0.25
smaller than 0.25
smaller than 0.25
smaller than 0.25
bigger than 0.25
smaller than 0.25
smaller than 0.25
smaller than 0.25
bigger than 0.25
smaller than 0.25
smaller than 0.25
smaller than 0.25
bigger than 0.25
smaller than 0.25
smaller than 0.25
bigger than 0.25
bigger than 0.25
bigger than 0.25
smaller than 0.25
smaller than 0.25
smaller than 0.25
bigger than 0.25
smaller than 0.25
smaller than 0.25
bigger than 0.25
smaller than 0.25
smaller than 0.25
smaller than 0.25
smaller than 0.25
smaller t

KeyboardInterrupt: 

In [54]:
id=pd.Series([])
j=0
for i in test_id:
    id[j]=i
    j=j+1
len(id)

/var/folders/0_/jlz3w3sd0rvgpp49r0h93j_c0000gn/T/ipykernel_66519/3084436451.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  id=pd.Series([])


144844

In [55]:
data=pd.DataFrame()

In [56]:
id[0]

130231

In [57]:
data.insert(0,"order_id",id)

In [58]:
data.head()

,order_id
0,130231
1,130232
2,130233
3,130234
4,130235


In [59]:
scores=pd.Series([])
j=0
for i, predicted in enumerate(ans):
    if predicted[0] > 0.018:
        scores[j]=1
    else:
        scores[j]=0
    j=j+1

/var/folders/0_/jlz3w3sd0rvgpp49r0h93j_c0000gn/T/ipykernel_66519/1203712447.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  scores=pd.Series([])


In [62]:
scores[1]

1

In [63]:
data.insert(1,"cancelled",scores)

In [65]:
data.to_csv("model1_1.csv")